In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
pip install wittgenstein

In [6]:
import wittgenstein

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/prediction churn of customers/PS_20174392719_1491204439457_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [8]:
def des_ana(dataframe):
    print('-----------------------Shape--------------------------')
    print(f'{dataframe.shape}\n\n')
    print('-----------------------Info--------------------------')
    print(f'{dataframe.info()}\n\n')
    print('-----------------------Columns--------------------------')
    print(f'{dataframe.columns}\n\n')
    print('-----------------------Descritive--------------------------')
    print(f'{dataframe.describe()}\n\n')
    print('-----------------------Null Values--------------------------')
    print(f'{dataframe.isnull().sum()}\n\n')
    print('-----------------------Unique Values--------------------------')
    print(f'{dataframe.nunique()}\n\n')

des_ana(df)

-----------------------Shape--------------------------
(6362620, 11)


-----------------------Info--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB
None


-----------------------Columns--------------------------
Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')


-----------------------Descritive--------------------------
 

In [9]:
df['isFraud'].value_counts()

0    6362604
1         16
Name: isFlaggedFraud, dtype: int64

# Handling data

In [10]:
# Dropping 'name'
df.drop(['nameOrig','nameDest'], axis=1, inplace=True)

In [11]:
cols = ['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']

scale_cols=['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']

# Scaling and Encoding data
ct = ColumnTransformer([
    ('scale', StandardScaler(), ['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']),
    ], remainder='passthrough'
)

df[scale_cols] = ct.fit_transform(df[scale_cols])

In [12]:
cat_cols = ['step','type','isFraud','isFlaggedFraud']

le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

In [13]:
df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,0,3,-0.281560,-0.229810,-0.237622,-0.323814,-0.333411,0,0
1,0,3,-0.294767,-0.281359,-0.285812,-0.323814,-0.333411,0,0
2,0,4,-0.297555,-0.288654,-0.292442,-0.323814,-0.333411,1,0
3,0,1,-0.297555,-0.288654,-0.292442,-0.317582,-0.333411,1,0
4,0,3,-0.278532,-0.274329,-0.282221,-0.323814,-0.333411,0,0


In [14]:
X = df.drop(['isFraud'], axis=1).values
y = df['isFraud'].values

In [15]:
# bal = RandomOverSampler()
# X1, y1 = bal.fit_resample(X1, y1)
# X2, y2 = bal.fit_resample(X2, y2)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [17]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906596
           1       0.86      0.97      0.91      2190

    accuracy                           1.00   1908786
   macro avg       0.93      0.99      0.96   1908786
weighted avg       1.00      1.00      1.00   1908786



In [18]:
from wittgenstein import RIPPER

rp = RIPPER()
rp.fit(X_train, y_train)
y_pred = rp.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00   1907055
        True       0.56      0.79      0.66      1731

    accuracy                           1.00   1908786
   macro avg       0.78      0.90      0.83   1908786
weighted avg       1.00      1.00      1.00   1908786



In [19]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1907470
           1       0.50      0.93      0.65      1316

    accuracy                           1.00   1908786
   macro avg       0.75      0.96      0.82   1908786
weighted avg       1.00      1.00      1.00   1908786



In [20]:
# from sklearn.model_selection import GridSearchCV

# seed = 42
# scores = []

# model_params = {
#     'XGBoost':{
#         'model': XGBClassifier(random_state=seed),
#         'params':{
#             'n_estimators':[200,300,500],
#             'max_depth':[5,10,15],
#             'learning_rate':[0.01,0.05]
#         }
#     },
#     'AdaBoost':{
#         'model': AdaBoostClassifier(random_state=seed),
#         'params':{
#             'n_estimators':[200,300,500],
#             'learning_rate':[0.01,0.05]
#         }
#     },
#     'RIPPER':{
#         'model': RIPPER(random_state=seed),
#         'params':{
#             'dl_allowance':[64,128,512],
#             'max_rule_conds':[3,5,8]
#         }
#     }
# }

In [21]:
# for model_name, mp in model_params.items():
#     clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(X, y)
#     scores.append({
#         'model':model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_
#     })

In [22]:
# results_train = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
# results_train

In [ ]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(RIPPER(random_state=42),n_estimators=10 ,random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_pred, y_test))